In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline

# Assignment - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:

def get_state_town():
    state_town_list = []
    with open('university_towns.txt') as f:
        for line in f:
            r = line
            if 'edit' in r:
                key = r.split('[')[0].strip()
            else:
                state_town_list.append([key,r.split('(')[0].strip()])
    return state_town_list

In [5]:
get_state_town()

[['Alabama', 'Auburn'],
 ['Alabama', 'Florence'],
 ['Alabama', 'Jacksonville'],
 ['Alabama', 'Livingston'],
 ['Alabama', 'Montevallo'],
 ['Alabama', 'Troy'],
 ['Alabama', 'Tuscaloosa'],
 ['Alabama', 'Tuskegee'],
 ['Alaska', 'Fairbanks'],
 ['Arizona', 'Flagstaff'],
 ['Arizona', 'Tempe'],
 ['Arizona', 'Tucson'],
 ['Arkansas', 'Arkadelphia'],
 ['Arkansas', 'Conway'],
 ['Arkansas', 'Fayetteville'],
 ['Arkansas', 'Jonesboro'],
 ['Arkansas', 'Magnolia'],
 ['Arkansas', 'Monticello'],
 ['Arkansas', 'Russellville'],
 ['Arkansas', 'Searcy'],
 ['California', 'Angwin'],
 ['California', 'Arcata'],
 ['California', 'Berkeley'],
 ['California', 'Chico'],
 ['California', 'Claremont'],
 ['California', 'Cotati'],
 ['California', 'Davis'],
 ['California', 'Irvine'],
 ['California', 'Isla Vista'],
 ['California', 'University Park, Los Angeles'],
 ['California', 'Merced'],
 ['California', 'Orange'],
 ['California', 'Palo Alto'],
 ['California', 'Pomona'],
 ['California', 'Redlands'],
 ['California', 'Rivers

In [6]:
import pandas as pd

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return pd.DataFrame(get_state_town(),columns=["State", "RegionName"])
    


In [7]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_gdplvl = pd.read_excel('gdplev.xls',skiprows=[0,1,2,3,4])
    quarterly = df_gdplvl.iloc[:,4:-1]
    quarterly.columns = ['Yearquarter'] + quarterly.columns[1:].tolist()
    quarterly = quarterly.dropna().reset_index(drop=True)
    quarterly = quarterly[quarterly['Yearquarter'] > '2000']
    quart_qdp = quarterly.iloc[:,2].tolist()
    for i,v in enumerate(quart_qdp):
        window = quart_qdp[i-1:i+4]
        if len(window) == 5:
            a,b,c,d,e = window
            if a > b and b > c and c <d and d < e :
                return quarterly.iloc[i,0]

In [9]:
get_recession_start()

'2009q1'

In [17]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df_gdplvl = pd.read_excel('gdplev.xls',skiprows=[0,1,2,3,4])

    quarterly = df_gdplvl.iloc[:,4:-1]

    quarterly.columns = ['Yearquarter'] + quarterly.columns[1:].tolist()

    quarterly = quarterly.dropna().reset_index(drop=True)
    quarterly = quarterly[quarterly['Yearquarter'] > '2000']
    quart_qdp = quarterly.iloc[:,2].tolist()
    for i,v in enumerate(quart_qdp):
        window = quart_qdp[i-1:i+4]
        if len(window) == 5:
            a,b,c,d,e = window
            if a > b and b > c and c <d and d < e :
                return quarterly.iloc[i+3,0]

In [18]:
get_recession_end()

'2009q4'

In [19]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df_gdplvl = pd.read_excel('gdplev.xls',skiprows=[0,1,2,3,4])


    quarterly = df_gdplvl.iloc[:,4:-1]

    quarterly.columns = ['Yearquarter'] + quarterly.columns[1:].tolist()

    quarterly = quarterly.dropna().reset_index(drop=True)
    quarterly = quarterly[quarterly['Yearquarter'] > '2000']
    quart_qdp = quarterly.iloc[:,2].tolist()
    for i,v in enumerate(quart_qdp):
        window = quart_qdp[i-1:i+4]
        if len(window) == 5:
            a,b,c,d,e = window
            if a > b and b > c and c <d and d < e :
                return quarterly.iloc[i+1,0]

In [20]:
get_recession_bottom()

'2009q2'

In [21]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv',encoding='latin-1')
    df_housing['State'] = df_housing['State'].apply(lambda x:states[x])
    columns = df_housing.columns.tolist()
    columns = [i for i,v in enumerate(columns) if (v > '2000' and v < '2016-10') or '-' not in v]
    df_housing = df_housing.iloc[:,columns].copy()
    df_housing = df_housing.melt(
        id_vars=df_housing.columns[:df_housing.columns.tolist().index('2000-01')],
        value_vars=df_housing.columns[df_housing.columns.tolist().index('2000-01'):],
        var_name='date',
        value_name='Price',
        col_level=None,
    )
    df_housing['date'] = pd.to_datetime(df_housing.date).dt.year.astype(str)\
                        + "q"\
                        + pd.to_datetime(df_housing.date).dt.quarter.astype(str)
    df_housing = df_housing.pivot_table(
        index=['State','RegionName'],
        columns='date',
        values='Price',
        aggfunc='mean')
    
    return df_housing

Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)




In [22]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    quartered = convert_housing_data_to_quarters()
    indexer =  get_list_of_university_towns().set_index(['State','RegionName'])
    university_prices = quartered[quartered.index.isin(indexer.index)].copy()
    university_prices['Price Ratio'] = university_prices[get_recession_start()]\
                                /university_prices[get_recession_bottom()]
    non_university_prices = quartered[~quartered.index.isin(indexer.index)].copy()
    non_university_prices['Price Ratio'] = non_university_prices[get_recession_start()]\
                                    /non_university_prices[get_recession_bottom()]
    x = ttest_ind(university_prices['Price Ratio'].dropna(),non_university_prices['Price Ratio'].dropna())
    
    if x[1] < 0.01:
        different = True
    else:
        different = False
    p = x[1]
    if university_prices['Price Ratio'].mean() < non_university_prices['Price Ratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
        
    return different,p,better
    


In [23]:
run_ttest()

(True, 0.00023810317796526157, 'university town')